In [3]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import gc

from tqdm import tqdm
from prj.config import DATA_DIR


BASE_PATH = DATA_DIR 

old = pl.read_parquet(BASE_PATH / 'partition_id=0' / 'part-0.parquet').with_columns(pl.lit(0).alias('part')).sort('date_id', 'time_id', 'symbol_id')
limit=100
fill_value=0.
features = [col for col in old.columns if col not in ['date_id', 'time_id', 'symbol_id', 'part']]
for partition in tqdm(range(1, 10)):
    curr = pl.read_parquet(BASE_PATH / f'partition_id={partition}' / 'part-0.parquet').with_columns(pl.lit(partition).alias('part')).sort('date_id', 'time_id', 'symbol_id')
    old.vstack(curr).with_columns(
        pl.col(features).fill_nan(None).forward_fill(limit=limit).fill_null(fill_value).name.keep()
    ).filter(
        pl.col('part') == partition
    ).drop('part')\
    .write_parquet(BASE_PATH / f'partition_id={partition}' / 'part-0_ffill.parquet')
    
    del old
    gc.collect()
    old = curr

# ffill first partition
pl.read_parquet(BASE_PATH / 'partition_id=0' / 'part-0.parquet').with_columns(
        pl.col(features).fill_nan(None).forward_fill(limit=limit).fill_null(fill_value).name.keep()
).write_parquet(BASE_PATH / 'partition_id=0' / 'part-0_ffill.parquet')

100%|██████████| 9/9 [01:22<00:00,  9.18s/it]
